In [4]:
import sys
import numpy as np
import keras
from keras.utils import Sequence
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import os
import imgaug as ia
from imgaug import augmenters as iaa
import cv2

In [13]:
BATCH_SIZE = 128
SEED = 777
SHAPE = (128, 128, 4)
DIR = 'E:/wujeccoai_data/human_atlas_data/all'
VAL_RATIO = 0.1 # 10 % as validation
THRESHOLD = 0.05 # due to different cost of True Positive vs False Positive, this is the probability threshold to predict the class as 'yes'

ia.seed(SEED)

In [14]:
def getTrainDataset():
    
    path_to_train = DIR + '/train/'
    data = pd.read_csv(DIR + '/train.csv')

    paths = []
    labels = []
    
    for name, lbl in zip(data['Id'], data['Target'].str.split(' ')):
        y = np.zeros(28)
        for key in lbl:
            y[int(key)] = 1
        paths.append(os.path.join(path_to_train, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

def getTestDataset():
    
    path_to_test = DIR + '/test/'
    data = pd.read_csv(DIR + '/sample_submission.csv')

    paths = []
    labels = []
    
    for name in data['Id']:
        y = np.ones(28)
        paths.append(os.path.join(path_to_test, name))
        labels.append(y)

    return np.array(paths), np.array(labels)


In [7]:
# credits: https://github.com/keras-team/keras/blob/master/keras/utils/data_utils.py#L302
# credits: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

class ProteinDataGenerator(keras.utils.Sequence):
            
    def __init__(self, paths, labels, batch_size, shape, shuffle = False, use_cache = False, augment = False):
        self.paths, self.labels = paths, labels
        self.batch_size = batch_size
        self.shape = shape
        self.shuffle = shuffle
        self.use_cache = use_cache
        self.augment = augment
        if use_cache == True:
            self.cache = np.zeros((paths.shape[0], shape[0], shape[1], shape[2]), dtype=np.float16)
            self.is_cached = np.zeros((paths.shape[0]))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]

        paths = self.paths[indexes]
        X = np.zeros((paths.shape[0], self.shape[0], self.shape[1], self.shape[2]))
        # Generate data
        if self.use_cache == True:
            X = self.cache[indexes]
            for i, path in enumerate(paths[np.where(self.is_cached[indexes] == 0)]):
                image = self.__load_image(path)
                self.is_cached[indexes[i]] = 1
                self.cache[indexes[i]] = image
                X[i] = image
        else:
            for i, path in enumerate(paths):
                X[i] = self.__load_image(path)

        y = self.labels[indexes]
                
        if self.augment == True:
            seq = iaa.Sequential([
                iaa.OneOf([
                    iaa.Fliplr(0.5), # horizontal flips
                    iaa.Crop(percent=(0, 0.1)), # random crops
                    # Small gaussian blur with random sigma between 0 and 0.5.
                    # But we only blur about 50% of all images.
                    iaa.Sometimes(0.5,
                        iaa.GaussianBlur(sigma=(0, 0.5))
                    ),
                    # Strengthen or weaken the contrast in each image.
                    iaa.ContrastNormalization((0.75, 1.5)),
                    # Add gaussian noise.
                    # For 50% of all images, we sample the noise once per pixel.
                    # For the other 50% of all images, we sample the noise per pixel AND
                    # channel. This can change the color (not only brightness) of the
                    # pixels.
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                    # Make some images brighter and some darker.
                    # In 20% of all cases, we sample the multiplier once per channel,
                    # which can end up changing the color of the images.
                    iaa.Multiply((0.8, 1.2), per_channel=0.2),
                    # Apply affine transformations to each image.
                    # Scale/zoom them, translate/move them, rotate them and shear them.
                    iaa.Affine(
                        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-180, 180),
                        shear=(-8, 8)
                    )
                ])], random_order=True)

            X = np.concatenate((X, seq.augment_images(X), seq.augment_images(X), seq.augment_images(X)), 0)
            y = np.concatenate((y, y, y, y), 0)
        
        return X, y
    
    def on_epoch_end(self):
        
        # Updates indexes after each epoch
        self.indexes = np.arange(len(self.paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        for item in (self[i] for i in range(len(self))):
            yield item
            
    def __load_image(self, path):
        R = Image.open(path + '_red.png')
        G = Image.open(path + '_green.png')
        B = Image.open(path + '_blue.png')
        Y = Image.open(path + '_yellow.png')

        im = np.stack((
            np.array(R), 
            np.array(G), 
            np.array(B),
            np.array(Y)), -1)
        
        im = cv2.resize(im, (SHAPE[0], SHAPE[1]))
        im = np.divide(im, 255)
        return im

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import metrics
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import keras
import tensorflow as tf

from tensorflow import set_random_seed
set_random_seed(SEED)

In [9]:
# credits: https://www.kaggle.com/guglielmocamporese/macro-f1-score-keras

def f1(y_true, y_pred):
    #y_pred = K.round(y_pred)
    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1-K.mean(f1)


In [10]:
# some basic useless model
def create_model(input_shape):
    
    dropRate = 0.25
    
    init = Input(input_shape)
    x = BatchNormalization(axis=-1)(init)
    x = Conv2D(8, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(8, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(16, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    c1 = Conv2D(16, (3, 3), padding='same')(x)
    c1 = ReLU()(c1)
    c2 = Conv2D(16, (5, 5), padding='same')(x)
    c2 = ReLU()(c2)
    c3 = Conv2D(16, (7, 7), padding='same')(x)
    c3 = ReLU()(c3)
    c4 = Conv2D(16, (1, 1), padding='same')(x)
    c4 = ReLU()(c4)
    x = Concatenate()([c1, c2, c3, c4])
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(32, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(dropRate)(x)
    #x = Conv2D(256, (1, 1), activation='relu')(x)
    #x = BatchNormalization(axis=-1)(x)
    #x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(28)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(0.1)(x)
    x = Dense(28)(x)
    x = Activation('sigmoid')(x)
    
    model = Model(init, x)
    
    return model

In [11]:
model = create_model(SHAPE)
model.compile(
    loss=f1_loss,
    optimizer=Adam(1e-03),
    metrics=['acc',f1])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 4)  0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 4)  16          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 126, 126, 8)  296         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 126, 126, 8)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_norm

max_pooling2d_5 (MaxPooling2D)  (None, 2, 2, 128)    0           batch_normalization_8[0][0]      
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 2, 2, 128)    0           max_pooling2d_5[0][0]            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 512)          0           dropout_5[0][0]                  
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 512)          0           flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 28)           14364       dropout_6[0][0]                  
__________________________________________________________________________________________________
re_lu_11 (

In [15]:
paths, labels = getTrainDataset()

# divide to 
keys = np.arange(paths.shape[0], dtype=np.int)  
np.random.seed(SEED)
np.random.shuffle(keys)
lastTrainIndex = int((1-VAL_RATIO) * paths.shape[0])

pathsTrain = paths[0:lastTrainIndex]
labelsTrain = labels[0:lastTrainIndex]
pathsVal = paths[lastTrainIndex:]
labelsVal = labels[lastTrainIndex:]

print(paths.shape, labels.shape)
print(pathsTrain.shape, labelsTrain.shape, pathsVal.shape, labelsVal.shape)

tg = ProteinDataGenerator(pathsTrain, labelsTrain, BATCH_SIZE, SHAPE, use_cache=True, augment = False, shuffle = False)
vg = ProteinDataGenerator(pathsVal, labelsVal, BATCH_SIZE, SHAPE, use_cache=True, shuffle = False)

# https://keras.io/callbacks/#modelcheckpoint
checkpoint = ModelCheckpoint('./base.model', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
reduceLROnPlato = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')

(31072,) (31072, 28)
(27964,) (27964, 28) (3108,) (3108, 28)


In [16]:
epochs = 200

use_multiprocessing = False # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 
workers = 1 # DO NOT COMBINE MULTIPROCESSING WITH CACHE! 

hist = model.fit_generator(
    tg,
    steps_per_epoch=len(tg),
    validation_data=vg,
    validation_steps=32,
    epochs=epochs,
    use_multiprocessing=use_multiprocessing,
    workers=workers,
    verbose=1,
    callbacks=[checkpoint])

Epoch 1/200


157/219 [====================>.........] - ETA: 34:45 - loss: 0.9188 - acc: 0.0234 - f1: 0.094 - ETA: 17:44 - loss: 0.9142 - acc: 0.0195 - f1: 0.097 - ETA: 12:03 - loss: 0.9140 - acc: 0.0208 - f1: 0.098 - ETA: 9:21 - loss: 0.9127 - acc: 0.0234 - f1: 0.098 - ETA: 9:08 - loss: 0.9129 - acc: 0.0266 - f1: 0.09 - ETA: 8:57 - loss: 0.9131 - acc: 0.0234 - f1: 0.09 - ETA: 8:51 - loss: 0.9122 - acc: 0.0234 - f1: 0.09 - ETA: 8:44 - loss: 0.9118 - acc: 0.0283 - f1: 0.09 - ETA: 8:39 - loss: 0.9121 - acc: 0.0321 - f1: 0.09 - ETA: 8:36 - loss: 0.9124 - acc: 0.0305 - f1: 0.09 - ETA: 8:31 - loss: 0.9119 - acc: 0.0312 - f1: 0.09 - ETA: 8:26 - loss: 0.9115 - acc: 0.0306 - f1: 0.09 - ETA: 8:22 - loss: 0.9108 - acc: 0.0312 - f1: 0.09 - ETA: 8:18 - loss: 0.9109 - acc: 0.0312 - f1: 0.09 - ETA: 8:14 - loss: 0.9110 - acc: 0.0307 - f1: 0.09 - ETA: 8:11 - loss: 0.9114 - acc: 0.0317 - f1: 0.09 - ETA: 8:06 - loss: 0.9118 - acc: 0.0317 - f1: 0.09 - ETA: 8:02 - loss: 0.9117 - acc: 0.0360 - f1: 0.09 - ETA: 7:59 - lo

219/219 [==============================] - ETA: 2:21 - loss: 0.8965 - acc: 0.1007 - f1: 0.10 - ETA: 2:18 - loss: 0.8964 - acc: 0.1011 - f1: 0.10 - ETA: 2:16 - loss: 0.8964 - acc: 0.1015 - f1: 0.10 - ETA: 2:14 - loss: 0.8963 - acc: 0.1023 - f1: 0.10 - ETA: 2:11 - loss: 0.8962 - acc: 0.1029 - f1: 0.10 - ETA: 2:09 - loss: 0.8962 - acc: 0.1034 - f1: 0.10 - ETA: 2:07 - loss: 0.8962 - acc: 0.1038 - f1: 0.10 - ETA: 2:05 - loss: 0.8961 - acc: 0.1042 - f1: 0.10 - ETA: 2:02 - loss: 0.8960 - acc: 0.1043 - f1: 0.10 - ETA: 2:00 - loss: 0.8959 - acc: 0.1047 - f1: 0.10 - ETA: 1:58 - loss: 0.8959 - acc: 0.1052 - f1: 0.10 - ETA: 1:55 - loss: 0.8958 - acc: 0.1057 - f1: 0.10 - ETA: 1:53 - loss: 0.8957 - acc: 0.1065 - f1: 0.10 - ETA: 1:51 - loss: 0.8956 - acc: 0.1072 - f1: 0.10 - ETA: 1:48 - loss: 0.8956 - acc: 0.1079 - f1: 0.10 - ETA: 1:46 - loss: 0.8955 - acc: 0.1083 - f1: 0.10 - ETA: 1:44 - loss: 0.8954 - acc: 0.1087 - f1: 0.10 - ETA: 1:41 - loss: 0.8953 - acc: 0.1091 - f1: 0.10 - ETA: 1:39 - loss: 0.8

157/219 [====================>.........] - ETA: 51s - loss: 0.8880 - acc: 0.2422 - f1: 0.100 - ETA: 51s - loss: 0.8843 - acc: 0.2188 - f1: 0.103 - ETA: 51s - loss: 0.8841 - acc: 0.2240 - f1: 0.104 - ETA: 51s - loss: 0.8823 - acc: 0.2305 - f1: 0.106 - ETA: 51s - loss: 0.8814 - acc: 0.2234 - f1: 0.107 - ETA: 50s - loss: 0.8814 - acc: 0.2214 - f1: 0.107 - ETA: 50s - loss: 0.8803 - acc: 0.2221 - f1: 0.107 - ETA: 50s - loss: 0.8805 - acc: 0.2178 - f1: 0.108 - ETA: 50s - loss: 0.8789 - acc: 0.2083 - f1: 0.109 - ETA: 49s - loss: 0.8798 - acc: 0.2070 - f1: 0.109 - ETA: 49s - loss: 0.8794 - acc: 0.2024 - f1: 0.109 - ETA: 49s - loss: 0.8788 - acc: 0.2044 - f1: 0.109 - ETA: 49s - loss: 0.8784 - acc: 0.2085 - f1: 0.109 - ETA: 49s - loss: 0.8781 - acc: 0.2093 - f1: 0.109 - ETA: 48s - loss: 0.8784 - acc: 0.2078 - f1: 0.109 - ETA: 48s - loss: 0.8784 - acc: 0.2075 - f1: 0.109 - ETA: 48s - loss: 0.8782 - acc: 0.2077 - f1: 0.109 - ETA: 48s - loss: 0.8779 - acc: 0.2070 - f1: 0.109 - ETA: 47s - loss: 0.87

219/219 [==============================] - ETA: 14s - loss: 0.8723 - acc: 0.2349 - f1: 0.113 - ETA: 14s - loss: 0.8722 - acc: 0.2351 - f1: 0.113 - ETA: 14s - loss: 0.8722 - acc: 0.2354 - f1: 0.113 - ETA: 13s - loss: 0.8722 - acc: 0.2354 - f1: 0.113 - ETA: 13s - loss: 0.8721 - acc: 0.2356 - f1: 0.113 - ETA: 13s - loss: 0.8721 - acc: 0.2358 - f1: 0.113 - ETA: 13s - loss: 0.8720 - acc: 0.2360 - f1: 0.113 - ETA: 12s - loss: 0.8719 - acc: 0.2363 - f1: 0.113 - ETA: 12s - loss: 0.8719 - acc: 0.2363 - f1: 0.114 - ETA: 12s - loss: 0.8718 - acc: 0.2364 - f1: 0.114 - ETA: 12s - loss: 0.8718 - acc: 0.2366 - f1: 0.114 - ETA: 12s - loss: 0.8717 - acc: 0.2365 - f1: 0.114 - ETA: 11s - loss: 0.8717 - acc: 0.2368 - f1: 0.114 - ETA: 11s - loss: 0.8717 - acc: 0.2365 - f1: 0.114 - ETA: 11s - loss: 0.8717 - acc: 0.2368 - f1: 0.114 - ETA: 11s - loss: 0.8716 - acc: 0.2367 - f1: 0.114 - ETA: 10s - loss: 0.8716 - acc: 0.2368 - f1: 0.114 - ETA: 10s - loss: 0.8715 - acc: 0.2371 - f1: 0.114 - ETA: 10s - loss: 0.87

...

219/219 [==============================] - ETA: 14s - loss: 0.6004 - acc: 0.4935 - f1: 0.397 - ETA: 14s - loss: 0.6000 - acc: 0.4939 - f1: 0.398 - ETA: 14s - loss: 0.5999 - acc: 0.4935 - f1: 0.398 - ETA: 14s - loss: 0.6000 - acc: 0.4933 - f1: 0.398 - ETA: 13s - loss: 0.5998 - acc: 0.4935 - f1: 0.398 - ETA: 13s - loss: 0.5999 - acc: 0.4939 - f1: 0.398 - ETA: 13s - loss: 0.6000 - acc: 0.4938 - f1: 0.398 - ETA: 13s - loss: 0.6000 - acc: 0.4940 - f1: 0.398 - ETA: 12s - loss: 0.5996 - acc: 0.4943 - f1: 0.398 - ETA: 12s - loss: 0.5996 - acc: 0.4939 - f1: 0.398 - ETA: 12s - loss: 0.5997 - acc: 0.4934 - f1: 0.398 - ETA: 12s - loss: 0.6000 - acc: 0.4931 - f1: 0.398 - ETA: 11s - loss: 0.6002 - acc: 0.4924 - f1: 0.397 - ETA: 11s - loss: 0.6004 - acc: 0.4925 - f1: 0.397 - ETA: 11s - loss: 0.6008 - acc: 0.4924 - f1: 0.397 - ETA: 11s - loss: 0.6006 - acc: 0.4922 - f1: 0.397 - ETA: 10s - loss: 0.6008 - acc: 0.4922 - f1: 0.397 - ETA: 10s - loss: 0.6009 - acc: 0.4925 - f1: 0.397 - ETA: 10s - loss: 0.60

157/219 [====================>.........] - ETA: 52s - loss: 0.5499 - acc: 0.5391 - f1: 0.438 - ETA: 52s - loss: 0.5945 - acc: 0.5156 - f1: 0.396 - ETA: 52s - loss: 0.5855 - acc: 0.5052 - f1: 0.418 - ETA: 52s - loss: 0.5775 - acc: 0.4961 - f1: 0.422 - ETA: 52s - loss: 0.5877 - acc: 0.5000 - f1: 0.408 - ETA: 52s - loss: 0.5781 - acc: 0.4974 - f1: 0.417 - ETA: 51s - loss: 0.5812 - acc: 0.5000 - f1: 0.414 - ETA: 51s - loss: 0.5835 - acc: 0.4932 - f1: 0.412 - ETA: 51s - loss: 0.5853 - acc: 0.4991 - f1: 0.412 - ETA: 51s - loss: 0.5906 - acc: 0.4977 - f1: 0.408 - ETA: 50s - loss: 0.5895 - acc: 0.5021 - f1: 0.407 - ETA: 50s - loss: 0.5951 - acc: 0.4961 - f1: 0.404 - ETA: 50s - loss: 0.5947 - acc: 0.4964 - f1: 0.406 - ETA: 50s - loss: 0.5946 - acc: 0.4961 - f1: 0.407 - ETA: 49s - loss: 0.5912 - acc: 0.4990 - f1: 0.409 - ETA: 49s - loss: 0.5921 - acc: 0.4980 - f1: 0.407 - ETA: 49s - loss: 0.5971 - acc: 0.4982 - f1: 0.403 - ETA: 49s - loss: 0.5972 - acc: 0.5013 - f1: 0.403 - ETA: 48s - loss: 0.60

219/219 [==============================] - ETA: 14s - loss: 0.6050 - acc: 0.4908 - f1: 0.390 - ETA: 14s - loss: 0.6049 - acc: 0.4903 - f1: 0.391 - ETA: 14s - loss: 0.6047 - acc: 0.4899 - f1: 0.391 - ETA: 14s - loss: 0.6048 - acc: 0.4897 - f1: 0.391 - ETA: 13s - loss: 0.6043 - acc: 0.4893 - f1: 0.391 - ETA: 13s - loss: 0.6043 - acc: 0.4894 - f1: 0.391 - ETA: 13s - loss: 0.6043 - acc: 0.4895 - f1: 0.391 - ETA: 13s - loss: 0.6046 - acc: 0.4893 - f1: 0.391 - ETA: 12s - loss: 0.6046 - acc: 0.4894 - f1: 0.391 - ETA: 12s - loss: 0.6045 - acc: 0.4894 - f1: 0.391 - ETA: 12s - loss: 0.6046 - acc: 0.4896 - f1: 0.391 - ETA: 12s - loss: 0.6042 - acc: 0.4894 - f1: 0.391 - ETA: 11s - loss: 0.6042 - acc: 0.4889 - f1: 0.391 - ETA: 11s - loss: 0.6040 - acc: 0.4886 - f1: 0.391 - ETA: 11s - loss: 0.6042 - acc: 0.4888 - f1: 0.391 - ETA: 11s - loss: 0.6043 - acc: 0.4891 - f1: 0.391 - ETA: 10s - loss: 0.6044 - acc: 0.4891 - f1: 0.391 - ETA: 10s - loss: 0.6043 - acc: 0.4892 - f1: 0.391 - ETA: 10s - loss: 0.60

157/219 [====================>.........] - ETA: 51s - loss: 0.6547 - acc: 0.5312 - f1: 0.336 - ETA: 51s - loss: 0.6412 - acc: 0.5156 - f1: 0.355 - ETA: 51s - loss: 0.6293 - acc: 0.4974 - f1: 0.365 - ETA: 51s - loss: 0.6253 - acc: 0.4785 - f1: 0.369 - ETA: 51s - loss: 0.6264 - acc: 0.4750 - f1: 0.369 - ETA: 51s - loss: 0.6222 - acc: 0.4714 - f1: 0.379 - ETA: 51s - loss: 0.6251 - acc: 0.4766 - f1: 0.380 - ETA: 50s - loss: 0.6248 - acc: 0.4805 - f1: 0.381 - ETA: 50s - loss: 0.6290 - acc: 0.4826 - f1: 0.376 - ETA: 50s - loss: 0.6280 - acc: 0.4883 - f1: 0.375 - ETA: 50s - loss: 0.6294 - acc: 0.4929 - f1: 0.372 - ETA: 50s - loss: 0.6276 - acc: 0.4896 - f1: 0.373 - ETA: 49s - loss: 0.6211 - acc: 0.4928 - f1: 0.378 - ETA: 49s - loss: 0.6238 - acc: 0.4950 - f1: 0.375 - ETA: 49s - loss: 0.6158 - acc: 0.4932 - f1: 0.382 - ETA: 49s - loss: 0.6065 - acc: 0.4946 - f1: 0.391 - ETA: 49s - loss: 0.6079 - acc: 0.4972 - f1: 0.390 - ETA: 48s - loss: 0.6055 - acc: 0.4974 - f1: 0.393 - ETA: 48s - loss: 0.60

219/219 [==============================] - ETA: 14s - loss: 0.6035 - acc: 0.4967 - f1: 0.394 - ETA: 14s - loss: 0.6036 - acc: 0.4967 - f1: 0.394 - ETA: 14s - loss: 0.6034 - acc: 0.4969 - f1: 0.394 - ETA: 14s - loss: 0.6032 - acc: 0.4967 - f1: 0.394 - ETA: 13s - loss: 0.6026 - acc: 0.4964 - f1: 0.395 - ETA: 13s - loss: 0.6025 - acc: 0.4965 - f1: 0.395 - ETA: 13s - loss: 0.6021 - acc: 0.4966 - f1: 0.395 - ETA: 13s - loss: 0.6024 - acc: 0.4964 - f1: 0.395 - ETA: 12s - loss: 0.6025 - acc: 0.4962 - f1: 0.395 - ETA: 12s - loss: 0.6028 - acc: 0.4966 - f1: 0.395 - ETA: 12s - loss: 0.6026 - acc: 0.4965 - f1: 0.395 - ETA: 12s - loss: 0.6024 - acc: 0.4967 - f1: 0.395 - ETA: 11s - loss: 0.6019 - acc: 0.4965 - f1: 0.395 - ETA: 11s - loss: 0.6021 - acc: 0.4964 - f1: 0.395 - ETA: 11s - loss: 0.6028 - acc: 0.4965 - f1: 0.395 - ETA: 11s - loss: 0.6028 - acc: 0.4966 - f1: 0.395 - ETA: 10s - loss: 0.6029 - acc: 0.4962 - f1: 0.394 - ETA: 10s - loss: 0.6030 - acc: 0.4959 - f1: 0.394 - ETA: 10s - loss: 0.60